## 바텀 라인의 영향력 분석

In [16]:
import pandas as pd
import requests
from urllib.parse import quote as quotekr # 한글 아이디를 인코딩 parse.quote
import time

In [17]:
api_key = 'RGAPI-547e5336-c2d4-4db0-8298-1bc1b0f5d74a'
request_header = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36",
                "Accept-Language": "ko,en-US;q=0.9,en;q=0.8,es;q=0.7",
                "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
                "Origin": "https://developer.riotgames.com",
                "X-Riot-Token": api_key
                }

#### 티어별 유저 5명 선별
* op.gg에서 무작위 추출
* Most top 3 챔피언이 모두 원거리 딜러
* 랭크 선호 포지션이 바텀 비율이 월등히 높은 사람
* 솔로랭크 50판 이상

In [ ]:
grand_master_user = ['줄라이77', '바텀개', 'always crying', '침대위섹시카이팅', '미드KING']
diamond_user = ['세혀닝', '강한남자한만윤', 'FEARN0T', '잘해봐요괜찮아요', '쿠로미짱이현']
# emerald_user = ['정의는 승리한다v', 'no return0', 'Cigar3', '주희보고싶은민교', 'Asnm']
emerald_user = ['정의는 승리한다v', 'no return0', 'Cigar3', '김잼니', 'Asnm']
platinum_user = ['협곡의 맙소사', '엄딴식', '지구뚝배기', '08년생 큐티정민', 'Gucci Vs']
gold_user = ['루픠는못말려', '홍구형', '카이팅의 귀재', 'Orangerie', '쥐파티']

grand_master_puuid = []
diamond_puuid = []
emerald_puuid = []
platinum_puuid = []
gold_puuid = []

In [ ]:
def user_puuid(users, puuids):
    for user in users:
        url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + quotekr(user)
        response = requests.get(url, headers=request_header)
        
        if response.status_code == 200: # Success
            pass

        elif response.status_code == 429: # Rate limit exceeded
            print('Rate limit exceeded error, api cost full')
            start_time = time.time()

            while True:
                if response.status_code == 200:
                    print('total wait time : ', time.time() - start_time)
                    print('recovery api cost')
                    break
                else:
                    print('try 3 second wait time')
                    time.sleep(3)
                    response = requests.get(url, headers=request_header)

        elif response.status_code == 503: # Service unavailable
            print('Service unavailable error')
            start_time = time.time()

            while True:
                if response.status_code == 200:
                    print('total wait time : ', time.time() - start_time)
                    print('recovery api cost')
                    break
                else:
                    print('try 5 second wait time')
                    time.sleep(5)
                    response = requests.get(url, headers=request_header)

        elif response.status_code == 403: # Forbidden
            print('Forbidden error, you need api renewal')
            print('break')
            return

        puuids.append(response.json()['puuid'])

In [ ]:
# def user_puuid2(users, puuids):
#     for user in users:
#         url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + quotekr(user)
#         response = requests.get(url, headers=request_header)

#         if response.status_code == 200:
#             pass
        
#         elif response.status_code == 429:
#             print("sleep", user, ' ', response.headers['Retry-After'])
#             time.sleep(int(response.headers['Retry-After']) + 5)
#             response = requests.get(url, headers=request_header)

#         puuids.append(response.json()['puuid'])

In [ ]:
user_puuid(grand_master_user, grand_master_puuid)
user_puuid(diamond_user, diamond_puuid)
user_puuid(emerald_user, emerald_puuid)
user_puuid(platinum_user, platinum_puuid)
user_puuid(gold_user, gold_puuid)

In [ ]:
grand_master_df = pd.DataFrame({
    'tier': ['grand_master' for i in range(5)],
    'summoner': grand_master_user,
    'puuid': grand_master_puuid
    })

diamond_df = pd.DataFrame({
    'tier': ['diamond' for i in range(5)],
    'summoner': diamond_user,
    'puuid': diamond_puuid
    })

emerald_df = pd.DataFrame({
    'tier': ['emerald' for i in range(5)],
    'summoner': emerald_user,
    'puuid': emerald_puuid
    })

platinum_df = pd.DataFrame({
    'tier': ['platinum' for i in range(5)],
    'summoner': platinum_user,
    'puuid': platinum_puuid
    })

gold_df = pd.DataFrame({
    'tier': ['gold' for i in range(5)],
    'summoner': gold_user,
    'puuid': gold_puuid
    })

In [ ]:
df = pd.concat([grand_master_df, diamond_df, emerald_df, platinum_df, gold_df], ignore_index=True)
display(df.head(2))
display(df.tail(2))

In [6]:
df.to_excel('puuid_data2.xlsx', index=False)

In [83]:
df = pd.read_excel('puuid_data2.xlsx')
display(df.head(2))
display(df.tail(2))


,tier,summoner,puuid
0,grand_master,줄라이77,ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEW...
1,grand_master,바텀개,EXr7XpD4deA3djszdNHUGUOVNcRxMwC26KdDhgGE3fJk9H...


,tier,summoner,puuid
23,gold,Orangerie,lAoGAWR0v4y0QbDTOMFKjFv_LlKfh_ymsZYyHsLkrr8xGM...
24,gold,쥐파티,CFbEyz3fMIYwR8t9htV_iKMT0OZKNK6gw0KWLeTajCG1aw...


In [84]:
df.loc[10:15]

,tier,summoner,puuid
10,emerald,정의는 승리한다v,wXvN9TJlkwSje56uoPtofaIgEoVCUGD63Hewfjw0GjpHO0...
11,emerald,no return0,GqsVg3QvhFw--4L0ascFso9yMh0eKrz9FdKtCf_leW4UOm...
12,emerald,Cigar3,eS7tRJlBcG3HdFA9LH0zq1s1wXzwMHKRgnQFMWYvwsf0Fk...
13,emerald,김잼니,LiQXFaJ4mVIhWVm511xSCIs6MS86Dt_awK6InfFBVKYgjZ...
14,emerald,Asnm,ev4ifUpwp6BDTaM8r2UERk6flI2aqfprAfUHho3M8rPPey...
15,platinum,협곡의 맙소사,tehmv7dX7eC5ZTZZjrzVIsTMBCb7J07UVBSQjSYK-5PoUJ...


In [5]:
# 주희보고싶은민교
# 포지션 변경으로 적정 데이터 수집 불가, 소환사 변경
df.loc[13] = ['emerald', '김잼니', 'LiQXFaJ4mVIhWVm511xSCIs6MS86Dt_awK6InfFBVKYgjZ_Tt-Ldzc1z7m2Q5iee8Ooj-17HEqn0qg']
df.loc[10:15]

,tier,summoner,puuid
10,emerald,정의는 승리한다v,wXvN9TJlkwSje56uoPtofaIgEoVCUGD63Hewfjw0GjpHO0...
11,emerald,no return0,GqsVg3QvhFw--4L0ascFso9yMh0eKrz9FdKtCf_leW4UOm...
12,emerald,Cigar3,eS7tRJlBcG3HdFA9LH0zq1s1wXzwMHKRgnQFMWYvwsf0Fk...
13,emerald,김잼니,LiQXFaJ4mVIhWVm511xSCIs6MS86Dt_awK6InfFBVKYgjZ...
14,emerald,Asnm,ev4ifUpwp6BDTaM8r2UERk6flI2aqfprAfUHho3M8rPPey...
15,platinum,협곡의 맙소사,tehmv7dX7eC5ZTZZjrzVIsTMBCb7J07UVBSQjSYK-5PoUJ...


#### 데이터 수집
* '바텀 게임' 이라는 말이 줄기 시작한 13.6패치가 3월 중순
* 1주일에 랭크게임 5판 한다고 가정
* 현재 8월 말이므로 약 20주
* 100게임 이전 인당 10판

In [8]:
def find_partiId(mat_data_resp, puuid):
    return mat_data_resp.json()['metadata']['participants'].index(puuid)

In [15]:
def puuid_to_matchid(puuid):
    match_ids = []
    start = 120
    count = 40
    retry_count = 0

    while len(match_ids) < 10: # 10경기 추출
        matches_url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}'
        match_list_resp = requests.get(matches_url, headers=request_header)

        while match_list_resp.status_code == 429: # Rate limit exceed
            print(retry_count, " sleep match_list", match_list_resp.headers['Retry-After'])
            retry_count += 1
            time.sleep(int(match_list_resp.headers['Retry-After']) + 5)
            match_list_resp = requests.get(matches_url, headers=request_header)

        if match_list_resp.status_code == 200:
            for mat_id in match_list_resp.json():
                if len(match_ids) == 10:
                    break

                mat_url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + mat_id
                mat_data_resp = requests.get(mat_url, headers=request_header)

                while mat_data_resp.status_code == 429: # Rate limit exceed
                    print(retry_count, " sleep mat_data", mat_data_resp.headers['Retry-After'])
                    retry_count += 1
                    time.sleep(int(mat_data_resp.headers['Retry-After']) + 5)
                    mat_data_resp = requests.get(mat_url, headers=request_header)

                if mat_data_resp.status_code == 200:
                    if mat_data_resp.json()['info']['gameMode'] == 'CLASSIC' \
                        and len(mat_data_resp.json()['info']['teams'][0]['bans']) > 0 \
                        and mat_data_resp.json()['info']['participants'][0]['gameEndedInEarlySurrender'] == False \
                        and mat_data_resp.json()['info']['gameDuration'] > 1500 \
                        and mat_data_resp.json()['info']['participants'][find_partiId(mat_data_resp, puuid)]['teamPosition'] == 'BOTTOM':
                        # 소환사 협곡
                        # 밴이 있음
                        # 재시작 경기 x
                        # 15분 이상
                        # 해당 소환사가 바텀라인

                        match_ids.append(mat_id)
                        retry_count = 0

                else:
                    print('\n\terror ', mat_data_resp.status_code)
                    print('\t', puuid)
                    break

            start += count

        else:
            print('\n\terror ', match_list_resp.status_code)
            print('\t', match_ids)
            break

    return match_ids

In [10]:
col = [('match' + str(x)) for x in range(1, 11)]
df_puuid = pd.DataFrame(columns=col)
df_puuid

,match1,match2,match3,match4,match5,match6,match7,match8,match9,match10


In [11]:
display(df.head(2))
display(df.tail(2))

,tier,summoner,puuid
0,grand_master,줄라이77,ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEW...
1,grand_master,바텀개,EXr7XpD4deA3djszdNHUGUOVNcRxMwC26KdDhgGE3fJk9H...


,tier,summoner,puuid
23,gold,Orangerie,lAoGAWR0v4y0QbDTOMFKjFv_LlKfh_ymsZYyHsLkrr8xGM...
24,gold,쥐파티,CFbEyz3fMIYwR8t9htV_iKMT0OZKNK6gw0KWLeTajCG1aw...


In [16]:
for puuid in df['puuid']:
    matchid_list = puuid_to_matchid(puuid)
    df_mid = pd.DataFrame([matchid_list[::-1]], columns=col)
    df_puuid = pd.concat([df_puuid, df_mid], ignore_index=True)

0  sleep match_list 41
0  sleep mat_data 90
0  sleep mat_data 88
0  sleep mat_data 88
0  sleep mat_data 88
0  sleep mat_data 89
0  sleep mat_data 88
0  sleep mat_data 88
0  sleep mat_data 88
0  sleep mat_data 89


In [73]:
df_puuid2 = df_puuid.drop([0, 1, 2], axis=0).reset_index(drop=True)

In [78]:
df_puuid2.to_excel('df_puuid2.xlsx', index=False)

In [74]:
# 닉네임 변경한 유저
name_list = list(df['summoner'])
count = 0
flag = False
changed = []

for mid in df_puuid2['match1']:
    url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + mid
    response = requests.get(url, headers=request_header)
    for parti in response.json()['info']['participants']:
        if parti['summonerName'] in name_list:
            print(count, parti['summonerName'])
            count += 1
            flag = True
            break

    if flag == False:
        users = response.json()['info']['participants']
        bottom1 = users[3]
        bottom2 = users[8]

        for pud in list(df['puuid']):
            if bottom1['puuid'] == pud:
                previous_name = bottom1['summonerName']
                current_name = list(df[df['puuid'] == pud]['summoner'])[0]

                print('\t', previous_name, '->', current_name)
                changed.append([mid, previous_name, current_name])

            elif bottom2['puuid'] == pud:
                previous_name = bottom2['summonerName']
                current_name = list(df[df['puuid'] == pud]['summoner'])[0]

                print('\t', previous_name, '->', current_name)
                changed.append([mid, previous_name, current_name])
        
    flag = False


	 이게마리되니 -> 줄라이77
0 바텀개
1 always crying
2 침대위섹시카이팅
	 리미리미치치 -> 미드KING
3 세혀닝
	 나는 존재 그자체 -> 강한남자한만윤
4 FEARN0T
5 잘해봐요괜찮아요
6 쿠로미짱이현
7 정의는 승리한다v
	 내진가를나만알아 -> no return0
8 Cigar3
9 김잼니
10 Asnm
11 협곡의 맙소사
12 엄딴식
	 JSH빡빡이 -> 지구뚝배기
13 08년생 큐티정민
14 Gucci Vs
15 루픠는못말려
16 홍구형
	 바텀갱은 언제와 -> 카이팅의 귀재
17 Orangerie
18 쥐파티


In [75]:
changed

[['KR_6611771034', '이게마리되니', '줄라이77'],
 ['KR_6581173722', '리미리미치치', '미드KING'],
 ['KR_6340571266', '나는 존재 그자체', '강한남자한만윤'],
 ['KR_6638713091', '내진가를나만알아', 'no return0'],
 ['KR_6523800400', 'JSH빡빡이', '지구뚝배기'],
 ['KR_6607652805', '바텀갱은 언제와', '카이팅의 귀재']]

In [86]:
previous_name = []
flag = False

for summoner in list(df['summoner']):
    for mid, prev, curr in changed:
        if summoner == curr:
            previous_name.append(prev)
            flag = True

    if flag == False:
        previous_name.append(summoner)
    
    flag = False

print(previous_name)

['이게마리되니', '바텀개', 'always crying', '침대위섹시카이팅', '리미리미치치', '세혀닝', '나는 존재 그자체', 'FEARN0T', '잘해봐요괜찮아요', '쿠로미짱이현', '정의는 승리한다v', '내진가를나만알아', 'Cigar3', '김잼니', 'Asnm', '협곡의 맙소사', '엄딴식', 'JSH빡빡이', '08년생 큐티정민', 'Gucci Vs', '루픠는못말려', '홍구형', '바텀갱은 언제와', 'Orangerie', '쥐파티']


In [87]:
df.insert(2, 'previous_summoner_name', previous_name)
df

,tier,summoner,previous_summoner_name,puuid
0,grand_master,줄라이77,이게마리되니,ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEW...
1,grand_master,바텀개,바텀개,EXr7XpD4deA3djszdNHUGUOVNcRxMwC26KdDhgGE3fJk9H...
2,grand_master,always crying,always crying,wBQaUzCDHu1SyLD330DHqGeJeFo3PhDeaYeyjlgqyp7SG0...
3,grand_master,침대위섹시카이팅,침대위섹시카이팅,4ZSpLbeybSbDG-s52ZQSsxQdGvtQbKZdvf_B-UwZ3P1_DJ...
4,grand_master,미드KING,리미리미치치,xr50XOMRd-wO_P8iKDaPiz1G45o15o8atDxQq8LDqrLxQY...
5,diamond,세혀닝,세혀닝,Q5YNI7bq1mrMB00HaWSomEW-A52vhXj9THL3pFS6b4sxQn...
6,diamond,강한남자한만윤,나는 존재 그자체,fybIOHNUpFHR4TcPoC8vyhKxVrYsZqV9ornEHNqL3ajkIn...
7,diamond,FEARN0T,FEARN0T,oZ4b0WJY2fP71MYT8BOpzUBSxUHG7pBloo-aGMWYzS09Q3...
8,diamond,잘해봐요괜찮아요,잘해봐요괜찮아요,2r4JbtA47RsnOICJJ1VjKO8-ogfvy5QytzAHTmKOyolaLq...
9,diamond,쿠로미짱이현,쿠로미짱이현,c3jkQfup6vcyiaLtTT0a5IZlHgiF40sq0v1MvGE2Pm6hib...


In [88]:
df.to_excel('puuid_data3.xlsx', index=False)

In [12]:
df_puuid.to_excel('temp_df_puuid.xlsx', index=False)

In [91]:
df2 = pd.concat([df, df_puuid2], axis=1)
df2.head(2)

,tier,summoner,previous_summoner_name,puuid,match1,match2,match3,match4,match5,match6,match7,match8,match9,match10
0,grand_master,줄라이77,이게마리되니,ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEW...,KR_6611771034,KR_6611825559,KR_6611894390,KR_6613941315,KR_6613987729,KR_6614204138,KR_6619754667,KR_6621536984,KR_6621619421,KR_6622454112
1,grand_master,바텀개,바텀개,EXr7XpD4deA3djszdNHUGUOVNcRxMwC26KdDhgGE3fJk9H...,KR_6662878759,KR_6663816089,KR_6663918544,KR_6663968631,KR_6664044375,KR_6666840236,KR_6667098846,KR_6667275166,KR_6667372041,KR_6668819105


In [92]:
df2.to_excel('matchid_data2.xlsx', index=False)

In [3]:
df2 = pd.read_excel('matchid_data2.xlsx')

In [18]:
display(df2.head(2))
display(df2.tail(2))

,tier,summoner,previous_summoner_name,puuid,match1,match2,match3,match4,match5,match6,match7,match8,match9,match10
0,grand_master,줄라이77,이게마리되니,ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEW...,KR_6611771034,KR_6611825559,KR_6611894390,KR_6613941315,KR_6613987729,KR_6614204138,KR_6619754667,KR_6621536984,KR_6621619421,KR_6622454112
1,grand_master,바텀개,바텀개,EXr7XpD4deA3djszdNHUGUOVNcRxMwC26KdDhgGE3fJk9H...,KR_6662878759,KR_6663816089,KR_6663918544,KR_6663968631,KR_6664044375,KR_6666840236,KR_6667098846,KR_6667275166,KR_6667372041,KR_6668819105


,tier,summoner,previous_summoner_name,puuid,match1,match2,match3,match4,match5,match6,match7,match8,match9,match10
23,gold,Orangerie,Orangerie,lAoGAWR0v4y0QbDTOMFKjFv_LlKfh_ymsZYyHsLkrr8xGM...,KR_6645334623,KR_6645438319,KR_6645642623,KR_6645737787,KR_6645815920,KR_6646017774,KR_6646123704,KR_6646228214,KR_6646723879,KR_6648866954
24,gold,쥐파티,쥐파티,CFbEyz3fMIYwR8t9htV_iKMT0OZKNK6gw0KWLeTajCG1aw...,KR_6520585109,KR_6520660560,KR_6520672537,KR_6520682964,KR_6520714018,KR_6520829018,KR_6520855792,KR_6521685419,KR_6521708922,KR_6525295806


#### 경기 데이터 전처리

In [6]:
def make_base_dict(puuid, matchid):
    base_dict = {'puuid': puuid, 'matchid': matchid}
    
    return base_dict

In [7]:
def response_match(matchid):
    url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + matchid
    resp_ma = requests.get(url, headers=request_header)

    return resp_ma

In [8]:
# participant ID 찾기
### match 에서는 pid-1
### timeline 에서는 'pid'
def find_participantId(resp_ma, puuid):
    return resp_ma.json()['metadata']['participants'].index(puuid) + 1

In [9]:
def make_data_end_dict(resp_ma, parti_id):
    data_end = resp_ma.json()['info']['participants'][parti_id - 1]

    data_end_dict = {'win': data_end['win'],
                     'kills': data_end['kills'],
                     'deaths': data_end['deaths'],
                     'assists': data_end['assists'],
                     'bountyGold': data_end['challenges']['bountyGold'],
                     'turretKills': data_end['turretKills'],
                     'turretsLost': data_end['turretsLost']
                    }
    
    return data_end_dict

In [10]:
def response_timelines(matchid):
    url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + matchid + '/timeline'
    resp_tl = requests.get(url, headers=request_header)
    
    return resp_tl

In [11]:
def make_data_15_dict(resp_tl, parti_id):
    data_15 = resp_tl.json()['info']['frames'][15]['participantFrames'][str(parti_id)]

    data_15_dict = {'totalDamageDone15': data_15['damageStats']['totalDamageDone'],
                    'totalDamageDoneToChampions15': data_15['damageStats']['totalDamageDoneToChampions'],
                    'totalDamageTaken15': data_15['damageStats']['totalDamageTaken'],
                    'level15': data_15['level'],
                    'xp15': data_15['xp'],
                    'minionsKilled15': data_15['minionsKilled'],
                    'totalGold15': data_15['totalGold']
                    }
    
    return data_15_dict

In [12]:
def make_data_15_kda_dict(resp_tl, parti_id):
    data_15_kda_dict = {'kills15': 0, 'deaths15': 0, 'assists15': 0}

    for times in range(0, 16):
        events = resp_tl.json()['info']['frames'][times]['events']

        for i in range(len(events)):
            if list(events[i])[0] == 'assistingParticipantIds' and events[i]['type'] == 'CHAMPION_KILL':
                
                if parti_id == events[i]['killerId']:
                    data_15_kda_dict['kills15'] += 1

                if parti_id == events[i]['victimId']:
                    data_15_kda_dict['deaths15'] += 1

                if parti_id in events[i]['assistingParticipantIds']:
                    data_15_kda_dict['assists15'] += 1

    return data_15_kda_dict

In [13]:
match_cols = ['puuid', 'matchid', \
              'win', 'kills', 'deaths', 'assists', 'bountyGold', 'turretKills', 'turretsLost', \
              'totalDamageDone15', 'totalDamageDoneToChampions15', 'totalDamageTaken15', 'level15', 'xp15', 'minionsKilled15', 'totalGold15', \
              'kills15', 'deaths15', 'assists15']

In [14]:
df_match = pd.DataFrame(columns=match_cols)
df_match

,puuid,matchid,win,kills,deaths,assists,bountyGold,turretKills,turretsLost,totalDamageDone15,totalDamageDoneToChampions15,totalDamageTaken15,level15,xp15,minionsKilled15,totalGold15,kills15,deaths15,assists15


In [26]:
errorFlag = False

for i in range(len(df2)):
    puuid = list(df2.loc[[i]]['puuid'])[0]
    match_ids = list(df2.loc[i])[4:]
    
    for matchid in match_ids:
        base_dict = make_base_dict(puuid, matchid)
        
        resp_ma = response_match(matchid)
        while resp_ma.status_code == 429: # Rate limit exceed
            print("sleep(ma)", resp_ma.headers['Retry-After'])
            time.sleep(int(resp_ma.headers['Retry-After']) + 3)
            resp_ma = response_match(matchid)

        if resp_ma.status_code != 200:
            print("resp_ma error ", resp_ma.status_code)
            errorFlag = True
            break

        parti_id = find_participantId(resp_ma, puuid)
        data_end_dict = make_data_end_dict(resp_ma, parti_id)

        resp_tl = response_timelines(matchid)
        while resp_tl.status_code == 429: # Rate limit exceed
            print("sleep(tl)", resp_tl.headers['Retry-After'])
            time.sleep(int(resp_tl.headers['Retry-After']) + 3)
            resp_tl = response_match(matchid)

        if resp_tl.status_code != 200:
            print("resp_tl error ", resp_tl.status_code)
            errorFlag = True
            break

        data_15_dict = make_data_15_dict(resp_tl, parti_id)
        data_15_kda_dict = make_data_15_kda_dict(resp_tl, parti_id)

        base_dict.update(data_end_dict)
        base_dict.update(data_15_dict)
        base_dict.update(data_15_kda_dict)
        df_match = df_match.append(base_dict, ignore_index=True)

    time.sleep(3)

    if errorFlag == True:
        print("error occurred")
        break

sleep(ma) 62
sleep(ma) 61
sleep(ma) 61
sleep(ma) 60


In [27]:
display(df_match.head(2))
display(df_match.tail(2))

,puuid,matchid,win,kills,deaths,assists,bountyGold,turretKills,turretsLost,totalDamageDone15,totalDamageDoneToChampions15,totalDamageTaken15,level15,xp15,minionsKilled15,totalGold15,kills15,deaths15,assists15
0,ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEW...,KR_6611771034,True,19,2,5,0,5,3,70886,9620,5167,10,6813,150,8133,4,0,2
1,ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEW...,KR_6611825559,False,7,4,8,450,2,10,50995,9032,4973,9,5170,126,6097,2,2,3


,puuid,matchid,win,kills,deaths,assists,bountyGold,turretKills,turretsLost,totalDamageDone15,totalDamageDoneToChampions15,totalDamageTaken15,level15,xp15,minionsKilled15,totalGold15,kills15,deaths15,assists15
248,CFbEyz3fMIYwR8t9htV_iKMT0OZKNK6gw0KWLeTajCG1aw...,KR_6521708922,True,12,8,18,150,2,2,59583,8303,7429,9,5411,112,7223,4,3,6
249,CFbEyz3fMIYwR8t9htV_iKMT0OZKNK6gw0KWLeTajCG1aw...,KR_6525295806,True,14,9,7,1050,5,5,44972,6316,6773,8,4989,110,5457,1,2,1


In [28]:
df_match.to_excel('match_log_data.xlsx', index=False)

In [ ]:
df_match = pd.read_excel('match_log_data.xlsx')

#### 연습구간

In [ ]:
# parti_ID 찾기
def find_participantId(response, puuid):
    return response.json()['metadata']['participants'].index(puuid) + 1

In [ ]:
find_participantId(resp_tl, df2['puuid'][0])

In [ ]:
resp_tl.json()['info']['frames'][15]['participantFrames']['9']

In [ ]:
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['damageStats']['totalDamageDone']) # 15분까지 가한 피해량
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['damageStats']['totalDamageDoneToChampions']) # 15분까지 챔피언에게 가한 피해량
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['damageStats']['totalDamageTaken']) # 15분까지 받은 피해량
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['level']) # 15분 레벨
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['minionsKilled']) # 15분 미니언 처치 수
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['totalGold']) # 15분 골드 누적 획득량
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['xp']) # 15분 경험치 누적 획득량

# 게임 종료
print(response.json()['info']['participants'][8]['challenges']['bountyGold']) # 현상금
print(response.json()['info']['participants'][8]['kills']) # 킬
print(response.json()['info']['participants'][8]['deaths']) # 데스
print(response.json()['info']['participants'][8]['assists']) # 어시스트
print(response.json()['info']['participants'][8]['turretKills']) # 자신이 파괴한 포탑 수
print(response.json()['info']['participants'][8]['turretsLost']) # 전체 파괴된 포탑 수
print(response.json()['info']['participants'][8]['win']) # 승패 여부

In [ ]:
resp_tl.json()['info']['frames'][8]['events'][22].keys()

In [ ]:
x = resp_tl.json()['info']['frames'][8]['events']
for i in range(len(x)):
    if list(x[i])[0] == 'assistingParticipantIds':
        print(i, x[i]['assistingParticipantIds'], x[i]['killerId'], x[i]['victimId'])

        if 9 in x[i]['assistingParticipantIds']:
            print('\tassist')
        if 9 == x[i]['killerId']:
            print('\tkill')
        if 9 == x[i]['victimId']:
            print('\tdeath')

##### 테스트

In [ ]:
pud = df2['puuid'][0]
mid = df2['match1'][0]
basedict = {'puuid': pud, 'matchid': mid}

In [ ]:
url1 = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + mid
resp_end = requests.get(url1, headers=request_header)
part_id = find_participantId(resp_end, pud)

In [ ]:
part_id

In [ ]:
data_end = resp_end.json()['info']['participants'][part_id-1]
data_end_dict = {'win': data_end['win'],
                 'kills': data_end['kills'],
                 'deaths': data_end['deaths'],
                 'assists': data_end['assists'],
                 'bountyGold': data_end['challenges']['bountyGold'],
                 'turretKills': data_end['turretKills'],
                 'turretsLost': data_end['turretsLost']
                 }

In [ ]:
data_end_dict

In [ ]:
url_tl = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + mid + '/timeline'
resp_tl = requests.get(url_tl, headers=request_header)

In [ ]:
data_15 = resp_tl.json()['info']['frames'][15]['participantFrames'][str(part_id)]
data_15_dict = {'totalDamageDone15': data_15['damageStats']['totalDamageDone'],
                'totalDamageDoneToChampions15': data_15['damageStats']['totalDamageDoneToChampions'],
                'totalDamageTaken15': data_15['damageStats']['totalDamageTaken'],
                'level15': data_15['level'],
                'xp15': data_15['xp'],
                'minionsKilled15': data_15['minionsKilled'],
                'totalGold15': data_15['totalGold']
                }

In [ ]:
data_15_dict

In [ ]:
x = resp_tl.json()['info']['frames'][8]['events']
for i in range(len(x)):
    if list(x[i])[0] == 'assistingParticipantIds':
        print(i, x[i]['assistingParticipantIds'], x[i]['killerId'], x[i]['victimId'])

        if 9 in x[i]['assistingParticipantIds']:
            print('\tassist')
        if 9 == x[i]['killerId']:
            print('\tkill')
        if 9 == x[i]['victimId']:
            print('\tdeath')

In [ ]:
data_15_kda = {'kills15': 0, 'deaths15': 0, 'assists15': 0}

In [ ]:
for times in range(0, 16):
    events = resp_tl.json()['info']['frames'][times]['events']

    for i in range(len(events)):
        if list(events[i])[0] == 'assistingParticipantIds' and events[i]['type'] == 'CHAMPION_KILL':
            
            if part_id == events[i]['killerId']:
                data_15_kda['kills15'] += 1

            if part_id == events[i]['victimId']:
                data_15_kda['deaths15'] += 1

            if part_id in events[i]['assistingParticipantIds']:
                data_15_kda['assists15'] += 1

In [ ]:
data_15_kda

In [ ]:
basedict.update(data_end_dict)
basedict.update(data_15_dict)
basedict.update(data_15_kda)

In [ ]:
basedict

In [ ]:
df_match.append(basedict, ignore_index=True)

##### -----

In [ ]:
tempdf = df_match.copy()
tempdf


In [ ]:
temp_dict = {'win':True, 'kills':10, 'deaths':20, 'assists':30}
td2 = {'new': False}

In [ ]:
tempdf = pd.DataFrame(columns=['win', 'kills', 'deaths', 'assists', 'new'])

In [ ]:
for key, value in temp_dict.items():
    print(key, value)
    tempdf.append(temp_dict, ignore_index=True)

tempdf

In [ ]:
temp_dict.update(td2)

In [ ]:
temp_dict

In [ ]:
tempdf.append(temp_dict, ignore_index=True)

In [ ]:
requests.get('http://ddragon.leagueoflegends.com/cdn/13.17.1/data/en_US/item.json').json()['data']['1001']['image']['full']

In [ ]:
url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + df2['match1'][0] + '/timeline'
response = requests.get(url, headers=request_header)
printjs(response.json())

In [ ]:
response.json()['info'].keys()

In [ ]:
response.json()['info']['frames']

In [ ]:
temp = response.json()['info']
len(temp['frames'])

In [ ]:
for x in temp['frames']:
    del x['participantFrames']

In [ ]:
temp['frames']

#### 15분
* 서렌 가능 시간
* 포탑 방패가 사라지는 14분 + 이후 타워가 깨질 수 있는 시간 약 1분
* 챔피언 성능이 달라지는 1코어 이상(1.5코어) 아이템 소지

#### 10분?
* 다른 라이너의 큰 개입 없이 라인전이 이루어지는 시간
* 서포터까지도 충분히 궁극기를 배우고 교전에 적극 참여하기 충분한 시간
* 12~13분이면 라인전이 끝남, 15분은 거의 3번째 용